# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6c50a58d90>
├── 'a' --> tensor([[ 0.3220, -0.1523,  0.9319],
│                   [-0.9158, -2.7671, -1.8058]])
└── 'x' --> <FastTreeValue 0x7f6c50a58f40>
    └── 'c' --> tensor([[ 0.2504,  0.4837, -0.8629,  1.8426],
                        [ 0.6503, -1.4840, -1.4313,  0.6199],
                        [ 0.4505, -1.8656,  0.3988,  0.4475]])

In [4]:
t.a

tensor([[ 0.3220, -0.1523,  0.9319],
        [-0.9158, -2.7671, -1.8058]])

In [5]:
%timeit t.a

88 ns ± 0.0478 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6c50a58d90>
├── 'a' --> tensor([[-1.1616, -0.8258, -0.1871],
│                   [-0.9460,  0.8954, -1.0534]])
└── 'x' --> <FastTreeValue 0x7f6c50a58f40>
    └── 'c' --> tensor([[ 0.2504,  0.4837, -0.8629,  1.8426],
                        [ 0.6503, -1.4840, -1.4313,  0.6199],
                        [ 0.4505, -1.8656,  0.3988,  0.4475]])

In [7]:
%timeit t.a = new_value

90.2 ns ± 0.0722 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3220, -0.1523,  0.9319],
               [-0.9158, -2.7671, -1.8058]]),
    x: Batch(
           c: tensor([[ 0.2504,  0.4837, -0.8629,  1.8426],
                      [ 0.6503, -1.4840, -1.4313,  0.6199],
                      [ 0.4505, -1.8656,  0.3988,  0.4475]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3220, -0.1523,  0.9319],
        [-0.9158, -2.7671, -1.8058]])

In [11]:
%timeit b.a

79.1 ns ± 0.0542 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.7073, -1.7655,  1.6680],
               [-0.3990, -1.1398,  1.4880]]),
    x: Batch(
           c: tensor([[ 0.2504,  0.4837, -0.8629,  1.8426],
                      [ 0.6503, -1.4840, -1.4313,  0.6199],
                      [ 0.4505, -1.8656,  0.3988,  0.4475]]),
       ),
)

In [13]:
%timeit b.a = new_value

650 ns ± 0.185 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.07 µs ± 2.49 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.6 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

175 µs ± 2.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

175 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6c50a6fdf0>
├── 'a' --> tensor([[[ 0.3220, -0.1523,  0.9319],
│                    [-0.9158, -2.7671, -1.8058]],
│           
│                   [[ 0.3220, -0.1523,  0.9319],
│                    [-0.9158, -2.7671, -1.8058]],
│           
│                   [[ 0.3220, -0.1523,  0.9319],
│                    [-0.9158, -2.7671, -1.8058]],
│           
│                   [[ 0.3220, -0.1523,  0.9319],
│                    [-0.9158, -2.7671, -1.8058]],
│           
│                   [[ 0.3220, -0.1523,  0.9319],
│                    [-0.9158, -2.7671, -1.8058]],
│           
│                   [[ 0.3220, -0.1523,  0.9319],
│                    [-0.9158, -2.7671, -1.8058]],
│           
│                   [[ 0.3220, -0.1523,  0.9319],
│                    [-0.9158, -2.7671, -1.8058]],
│           
│                   [[ 0.3220, -0.1523,  0.9319],
│                    [-0.9158, -2.7671, -1.8058]]])
└── 'x' --> <FastTreeValue 0x7f6b67130ca0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.9 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6b671305e0>
├── 'a' --> tensor([[ 0.3220, -0.1523,  0.9319],
│                   [-0.9158, -2.7671, -1.8058],
│                   [ 0.3220, -0.1523,  0.9319],
│                   [-0.9158, -2.7671, -1.8058],
│                   [ 0.3220, -0.1523,  0.9319],
│                   [-0.9158, -2.7671, -1.8058],
│                   [ 0.3220, -0.1523,  0.9319],
│                   [-0.9158, -2.7671, -1.8058],
│                   [ 0.3220, -0.1523,  0.9319],
│                   [-0.9158, -2.7671, -1.8058],
│                   [ 0.3220, -0.1523,  0.9319],
│                   [-0.9158, -2.7671, -1.8058],
│                   [ 0.3220, -0.1523,  0.9319],
│                   [-0.9158, -2.7671, -1.8058],
│                   [ 0.3220, -0.1523,  0.9319],
│                   [-0.9158, -2.7671, -1.8058]])
└── 'x' --> <FastTreeValue 0x7f6b67130730>
    └── 'c' --> tensor([[ 0.2504,  0.4837, -0.8629,  1.8426],
                        [ 0.6503, -1.4840, -1.4313,  0.6199],
                 

In [23]:
%timeit t_cat(trees)

37.6 µs ± 353 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

76.4 µs ± 183 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2504,  0.4837, -0.8629,  1.8426],
                       [ 0.6503, -1.4840, -1.4313,  0.6199],
                       [ 0.4505, -1.8656,  0.3988,  0.4475]],
              
                      [[ 0.2504,  0.4837, -0.8629,  1.8426],
                       [ 0.6503, -1.4840, -1.4313,  0.6199],
                       [ 0.4505, -1.8656,  0.3988,  0.4475]],
              
                      [[ 0.2504,  0.4837, -0.8629,  1.8426],
                       [ 0.6503, -1.4840, -1.4313,  0.6199],
                       [ 0.4505, -1.8656,  0.3988,  0.4475]],
              
                      [[ 0.2504,  0.4837, -0.8629,  1.8426],
                       [ 0.6503, -1.4840, -1.4313,  0.6199],
                       [ 0.4505, -1.8656,  0.3988,  0.4475]],
              
                      [[ 0.2504,  0.4837, -0.8629,  1.8426],
                       [ 0.6503, -1.4840, -1.4313,  0.6199],
                       [ 0.4505, -1.8656,  0.3988,  0.4475]],

In [26]:
%timeit Batch.stack(batches)

98.6 µs ± 124 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2504,  0.4837, -0.8629,  1.8426],
                      [ 0.6503, -1.4840, -1.4313,  0.6199],
                      [ 0.4505, -1.8656,  0.3988,  0.4475],
                      [ 0.2504,  0.4837, -0.8629,  1.8426],
                      [ 0.6503, -1.4840, -1.4313,  0.6199],
                      [ 0.4505, -1.8656,  0.3988,  0.4475],
                      [ 0.2504,  0.4837, -0.8629,  1.8426],
                      [ 0.6503, -1.4840, -1.4313,  0.6199],
                      [ 0.4505, -1.8656,  0.3988,  0.4475],
                      [ 0.2504,  0.4837, -0.8629,  1.8426],
                      [ 0.6503, -1.4840, -1.4313,  0.6199],
                      [ 0.4505, -1.8656,  0.3988,  0.4475],
                      [ 0.2504,  0.4837, -0.8629,  1.8426],
                      [ 0.6503, -1.4840, -1.4313,  0.6199],
                      [ 0.4505, -1.8656,  0.3988,  0.4475],
                      [ 0.2504,  0.4837, -0.8629,  1.8426],
                   

In [28]:
%timeit Batch.cat(batches)

181 µs ± 907 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

449 µs ± 9.06 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
